In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as stats
import math
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.cluster.hierarchy import dendrogram, linkage
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
%matplotlib inline
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

In [ ]:
happiness_report = "world_happiness_report_2019_original_file.csv"
region="Country-names-with-region.csv"

In [ ]:
happiness_report_df = pd.read_csv(happiness_report)
region_df=pd.read_csv(region)

In [ ]:
happiness_report_df.head()

In [ ]:
region_df.head()

In [ ]:
happiness_df=happiness_report_df.merge(region_df, on='Country name', how='left')

In [ ]:
happiness_df.head()

In [ ]:
happiness_df.columns

In [ ]:
happiness_df[['Country name', 'Region', 'Year', 'Life Ladder', 'Log GDP per capita',
       'Social support', 'Healthy life expectancy at birth',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption', 'Positive affect', 'Negative affect',
       'Confidence in national government', 'Democratic Quality',
       'Delivery Quality', 'Standard deviation of ladder by country-year',
       'Standard deviation/Mean of ladder by country-year',
       'GINI index (World Bank estimate)',
       'GINI index (World Bank estimate), average 2000-16',
       'gini of household income reported in Gallup, by wp5-year',
       'Most people can be trusted, Gallup',
       'Most people can be trusted, WVS round 1981-1984',
       'Most people can be trusted, WVS round 1989-1993',
       'Most people can be trusted, WVS round 1994-1998',
       'Most people can be trusted, WVS round 1999-2004',
       'Most people can be trusted, WVS round 2005-2009',
       'Most people can be trusted, WVS round 2010-2014']]

In [ ]:
happy_df = happiness_df[['Country name', 'Region', 'Year', 'Life Ladder', 'Log GDP per capita',
       'Social support', 'Healthy life expectancy at birth',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption', 'Positive affect', 'Negative affect',
       'Confidence in national government', 'Democratic Quality',
       'Delivery Quality', 'Standard deviation of ladder by country-year',
       'Standard deviation/Mean of ladder by country-year',
       'GINI index (World Bank estimate)','gini of household income reported in Gallup, by wp5-year',
       'Most people can be trusted, Gallup']]
happy_df.head()

In [ ]:
report_df= happy_df.rename(columns={'Life Ladder':'Happiness_Score', 'Log GDP per capita':'GDP', 'Freedom to make life choices':'Freedom'})
report_df.head()

In [ ]:
report_df.dropna()

In [ ]:
report_df.columns

In [ ]:
df = report_df.apply (pd.to_numeric, errors='coerce')
df = report_df.dropna()
print (df)

In [ ]:
report_df.groupby('Region')['Happiness_Score','GDP'].mean().sort_values(by="Happiness_Score", ascending=False)

In [ ]:
def correlation_matrix(df):
    plt.figure(figsize=(14,12))
    sns.heatmap(df.corr(), center=0, annot=True, cmap="RdBu_r", linewidths = .5, fmt='.2f',annot_kws={'size': 10})   

In [ ]:
correlation_matrix(df)
plt.savefig("correlations.svg")

In [ ]:
# First we prepare the data to use a model
df1 = df.drop(["Country name","Region"], axis=1)
y = df1["Happiness_Score"]
X = df1.drop(["Happiness_Score"], axis=1)

In [ ]:
for feature in X.columns.values.tolist():
    X = df1.drop(["Happiness_Score"], axis=1)
    X = X[feature]
    X = X.values.reshape(-1,1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    plt.title("Regressions of " + feature)
    plt.scatter(X, y)

    degrees = [1,2,3,4]
    for degree in degrees:
        poly_features = PolynomialFeatures(degree=degree)

        # transforms the existing features to higher degree features.
        X_train_poly = poly_features.fit_transform(X_train)

        # fit the transformed features to Linear Regression
        poly_model = LinearRegression()
        poly_model.fit(X_train_poly, y_train)

        # predicting on test data-set
        y_test_predict = poly_model.predict(poly_features.fit_transform(X_test))

        # Plotting the regression

        new_x, new_y = zip(*sorted(zip(X_test, y_test_predict)))
        plt.plot(new_x, new_y, label="Degree = " + str(degree))

        plt.xlabel(feature)
        plt.ylabel("Happiness_Score")
        plt.legend()
        plt.savefig("start.svg")
    plt.show()
    plt.close()

These plots can show us how adjusted are the regressions with the data using only an indicator each time. Now we will try to use all the columns to predict the happiness score, but if we want to plot the regression result line we will need to perform a PCA to the data.

Applying PCA to our data we will obtain a single “feature” that will allow us to represent all the attribute columns in a single axis.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# First we prepare the data to use a model
reg_data = df.drop(["Country name","Region"], axis=1)
y = reg_data["Happiness_Score"]
X = reg_data.drop(["Happiness_Score"], axis=1)

# Normalisation and PCA
X_norm = StandardScaler().fit_transform(X)
pca = PCA(n_components=1)
X_pca = pca.fit_transform(X_norm)

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.33, random_state=42)

plt.scatter(X_pca, y)

degrees = [1,2,3,4]
for degree in degrees:
    poly_features = PolynomialFeatures(degree=degree)

    # transforms the existing features to higher degree features.
    X_train_poly = poly_features.fit_transform(X_train)

    # fit the transformed features to Linear Regression
    poly_model = LinearRegression()
    poly_model.fit(X_train_poly, y_train)

    # predicting on test data-set
    y_test_predict = poly_model.predict(poly_features.fit_transform(X_test))

    # Plotting the regression
    new_x, new_y = zip(*sorted(zip(X_test, y_test_predict)))
    plt.plot(new_x, new_y, label="Degree = " + str(degree))
    plt.xlabel("Principal Component")
    plt.ylabel("Happiness_Score")
    plt.legend()
    plt.savefig("PCA.svg")
plt.show()
plt.close()



In [ ]:
# evaluating the model on test dataset
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_predict))
r2_test = r2_score(y_test, y_test_predict)
print("RMSE test (PCA): " + str(rmse_test))
print("R2square test (PCA): " + str(r2_test))

The Rsquare is the proportion of the variance in the dependent variable that is predictable from the independent variable. Basically it says how good the regression function is fitting the data. The value goes between 0 and 1, being 1 the best value and 0 the worst.

RMSE is the standard deviation of the residuals (prediction errors). Residuals are a measure of how far from the regression line data points are; RMSE is a measure of how spread out these residuals are. In other words, it tells you how concentrated the data is around the line of best fit.

Talking about the performance of the regression models, the R-square of the showed models keeps around a 0.7 value. By looking at the distribution of the data on the previous charts, it can be observed that the regression functions are fitting the data properly.

There are difficult data distributions for a regression model to fit, as for example the one observed in the PCA chart. The more high the degree of the regression polynomial, the better fitted the data. Nonetheless, the increase of the degree of the model can cause overfitting, so this is an aspect that has to be treated carefully to build a correct regression model.